In [1]:
import os
import pandas as pd

In [51]:
alignment_folder = '/Users/eweisbar/Desktop/alignment/20240715_batch2/'
subfolders = os.listdir(alignment_folder)
df = pd.DataFrame()
for folder in subfolders:
    if os.path.exists(os.path.join(alignment_folder,folder,'Image.csv')):
        file = pd.read_csv(os.path.join(alignment_folder,folder,'Image.csv'))
        df = pd.concat([df,file])
print (len(df))

243


In [52]:
meta_cols = [x for x in df.columns if 'Metadata' in x]
shift_cols = [x for x in df.columns if 'Align' in x and 'DAPI' in x]
df = df[meta_cols+shift_cols].sort_values(by=meta_cols)
df = df.loc[df['Metadata_Cycle']!=1]

In [53]:
meandf = df.groupby(['Metadata_Plate','Metadata_Cycle','Metadata_Well']).mean(numeric_only =True).reset_index()
meandf = meandf.rename(columns={'Align_Xshift_DAPI':'Align_Xshift_DAPI_mean','Align_Yshift_DAPI':'Align_Yshift_DAPI_mean'})
df = df.merge(meandf, on=['Metadata_Plate','Metadata_Cycle','Metadata_Well'])
df['X_diff'] = (df['Align_Xshift_DAPI']-df['Align_Xshift_DAPI_mean']).abs()
df['Y_diff'] = (df['Align_Yshift_DAPI']-df['Align_Yshift_DAPI_mean']).abs()
df.loc[df['X_diff']>2, 'WARN'] = 'WARN'
df.loc[df['Y_diff']>2, 'WARN'] = 'WARN'
df['Plate_Well_Cycle'] = df['Metadata_Plate'] + '_' + df['Metadata_Well'] + '_' + df['Metadata_Cycle']
print (len(df.loc[df['WARN'] =='WARN','Plate_Well_Cycle'].unique()))
df.loc[df['WARN'] =='WARN','Plate_Well_Cycle'].unique()

33


array(['Plate56_A2_Cycle01', 'Plate56_B1_Cycle01', 'Plate67_B3_Cycle01',
       'Plate56_A2_Cycle02', 'Plate56_B1_Cycle02', 'Plate67_B3_Cycle02',
       'Plate56_A2_Cycle03', 'Plate56_B1_Cycle03', 'Plate67_B3_Cycle03',
       'Plate56_A2_Cycle04', 'Plate56_B1_Cycle04', 'Plate67_B3_Cycle04',
       'Plate56_A2_Cycle05', 'Plate56_B1_Cycle05', 'Plate67_B3_Cycle05',
       'Plate56_A2_Cycle06', 'Plate56_B1_Cycle06', 'Plate56_A1_Cycle06',
       'Plate67_B3_Cycle06', 'Plate56_A2_Cycle07', 'Plate56_B1_Cycle07',
       'Plate67_B3_Cycle07', 'Plate56_A2_Cycle08', 'Plate56_B1_Cycle08',
       'Plate56_A1_Cycle08', 'Plate67_B3_Cycle08', 'Plate56_A2_Cycle09',
       'Plate56_B1_Cycle09', 'Plate56_A1_Cycle09', 'Plate67_B3_Cycle09',
       'Plate56_A2_Cycle10', 'Plate56_B1_Cycle10', 'Plate67_B3_Cycle10'],
      dtype=object)